In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

Hi everyone! Today we are building *Scalebandit*, which will address the problems we encountered in the last lecture.

If you remember back to Lecture 2, we actually made *personalised content* by introducing the concept of <span style="color:blue">states</span>. We now had a row for each different type of user that we wanted to make the best YouTube homepage for. We also remember that this worked perfectly, until we tried to scale it. We discovered what is called the **curse of dimensionality**. As we got exited and added more <span style="color:blue">states</span>, our table blew up, because we had to have a single cell, for each new 'thing' we added, and if multiple things are going to have their own cell, then the total number of cells is all these things multiplied together. So the table exploded in size, becoming hard to fit into our computer memory, and also was really hard to get anything meaningful out of, because of the fact that we needed to actually have insane amounts of data to be sure that we saw each cell, for each specific case (tablet user in France at midnight), several times so our estmiate of that value became something that we could trust. 

This is the wall that the table methods hit. The brain works kind of by **memorisation**. It *needs a separate memory slot (a row in our table) for every single possible situation*. 

Today, we are going to solve this problem by trying to find a method that can **generalise** (i.e., say anything about things that it has never encountered before). 

---

Think about when you learned the gangetabell. One day your teacher said: "tomorrow I'm going to test you with numbers from 0 to 10, so you need to learn this". You go home and you write down a large 2D-table for all numbers that can multiply (e.g., 2 x 3 = 6, 4 x 5 = 20 etc.). So now each answer has it's own cell. Then you try rally hard to hold your hand over every cell and try again and again to remember what the answer is, for each combination of numbers, until you get it absolutely perfect, and you go with a big dick walk to school the next day. You know *all the answers*. This is a bit like what we are doing with our table in the last lecture. We had our table, and if you wanted to know the value of showing a viral thing to a new viewer, you just looked it up: `belief_table[0, 0]`. If the entry wasn't there, you had no idea of what the value was. 

However, there is another way. Imagine you instead built a device, that for any two numbers you put in, gave you the answer. So now when the teacher gave you the numbers 4 and 5, you used your device it gave you 20 back, which you passed on as your answer to the teacher.

This is exactly the conceptual change we are going to make today. We **want something that takes our state and gives back the value for showing each different thumbnail**. We don't want to remember it, we want to be able to calculate it on the fly. Even for states we haven't seen before.

Formally, we can say that we want: `values = f(state)`

Let's give this <span style="color:LightCoral">function</span> a name, so that we can talk about it. Let's call it our *Value function approximator*. Because we want the value, we use a device called a function to return it to us, and approximator because we give the thing a little slack, so that it doesn't have to be perfectly accurate at first, but we can kind of make a reasonable guess for *any* state, even ones we haven't seen before (and how could that be accurate?). This whole process, if we are able to be more and more accurate, is known as **generalisation**.


#### Environment


In [3]:
class HomepageEnv:
    def __init__(self):
        self.contexts = ["New viewer", "Subscriber"]
        self.actions = ["🔥 Viral thing", "🤖 AI tutorial", "💤 Boring politics debate"]
        self.num_contexts = len(self.contexts)
        self.num_actions = len(self.actions)

    def get_reward(self, context_idx, action_idx):
        # New viewers (context 0) are attracted by viral videos
        if context_idx == 0: # the index for a new viewer
            true_probs = [0.6, 0.1, 0.05] # gives the size of a spinning wheel, here "viral thing" has 60% of the space 
            return 1 if np.random.rand() < true_probs[action_idx] else 0 # like spinning a wheel that can land anywhere from 0.00 to 0.99
        # Subscribers (context 1) want deeper content
        elif context_idx == 1:
            true_probs = [0.1, 0.5, 0.7] # The subscribers are thoughtfull and use YouTube a bit differently, they want boring debates
            return 1 if np.random.rand() < true_probs[action_idx] else 0

# Initialize environment
env = HomepageEnv()

In [4]:
env.contexts

['New viewer', 'Subscriber']

In [5]:
env.actions

['🔥 Viral thing', '🤖 AI tutorial', '💤 Boring politics debate']


#### Building: the device


In [7]:
# what's the simplest possible function that we can build? Let's try just a linear thing
# So, let's represent our context as a list
new_viewer = [1, 0] # just a one representing the type of person that has arrived at our YouTube page
subscriber = [0, 1] # the same for this type of user

In [10]:
# What do we miss now? If you remember back to our machine learning class, we actually know how to build this
w = np.random.randn(env.num_contexts, env.num_actions)
w

array([[-1.91990729,  0.76001969,  0.34067624],
       [-0.09454043,  0.49068807,  0.67292079]])

In [11]:
# We can now estiamte the value, by taking our weights and multiply them with our featues
np.array(new_viewer) @ w     # (1, 2) x (2, 3) -> (1, 3)

array([-1.91990729,  0.76001969,  0.34067624])

In [12]:
# In the same way, we can estimate the values from our subscribers
np.array(subscriber) @ w

array([-0.09454043,  0.49068807,  0.67292079])

So, by estimating our values in this manner, you see that we 'pluck out' the rows for that type of person we are actually interested in, by dot-producting with a feature vector that is 1, if this is the user we are about, and 0 otherwise. 

In [10]:
# so all that we need from our state representation is now 
state = np.zeros(env.num_contexts)
state

array([0., 0.])

In [11]:
# so, a user shows up
np.random.randint(0, env.num_contexts) # get a random number between 0 and 1

0

In [12]:
# and we set the feature to be 1, for this type of user that has showed up
state[np.random.randint(0, env.num_contexts)] = 1
state

array([0., 1.])

In [130]:
# now to get the estimates of what we should how to this user, we can just get the correct row by the dot product
correct_row_with_probs = state @ w
correct_row_with_probs

array([-0.63579846,  0.39491446, -1.471798  ])

In [132]:
np.argmax(correct_row_with_probs).item()

1

Let's forget about all the details for a moment and focus on one **single event**.

Imagine we see a new viewer (context = [1, 0]). It uses it's current dials to calcualte the estimates CTRs an gets:
y_pred = [0.4, 0.1, 0.2]

Based on this, it chooses the best action (as we have seen in the eprevious lecture, with the argmax), so `action_idx = 0` because in the list 0.4 has the highest value, and it is located at index 0. We then show the '🔥 Viral thing' thumbnail and the user clicks! The agent recieves a **real-world result**: `reward = 1`

So, now we are faced with a simple fact:

1. Our dials was set up such that we thought the click-through rate was 0.4
2. the reality was better than this, we got 1.

So, the difference between reality and what we thought the reality was: `reality - y_pred = 1 - 0.4 = 0.6`. This is our error.

In some sense, it was too pessimistic. How can we adjust our <span style="color:purple">dials</span> (*w*) so that if we find ourself in this exact situation again, our thinking will be more like what we have actually experienced now? I.e., a little bit closer to 1?

Well, we actually know this from our ML course! Supervised learning! Gradient descent! WOOOHO :))

So, we know that to nudge the weight in a direction that reduces our error, we need to multiply by the gradient of our function approximator. So, what is the gradient? Well, our function approximator is pretty easy since it is just linear in the weights, so we have: y_pred = s @ w -> derror/dw = **s**. So our gradient is just the <span style="color:blue">state</span> list. 

So, we use supervised learning. So our update rule becomes
<div style="border: 2px solid #988558; padding: 10px; border-radius: 8px; background-color: #EADDCA;">
<b>w_new </b> <-- old_w + little_bit * error * s
</div>


And this little bit is called the 'learning rate'. This just mean we don't want to step too far, risking overshooting our target.

See that we introduced a new word there: the <span style="color:goldenrod">target</span>. What do we mean by target? Well, the <span style="color:aquamarine">environment</span> gives back sort of a "reality check". We just call this reality check *something*, and that is the target. 

Remember from supervised learning, our estimate of the best click-through-rate (CTR) now fully depends on W. These are, like in supervised learning dials we can turn now, and if we are lucky, potentially get them to some values that is right no matter what. 

However, we don't know in advanve what these values *should be*, so we need to **train those dials up** to give us the <span style="color:green">right answer</span>. And the *way we do that is where the <span style="color:coral">reinforcement learning</span>* comes in. 


#### Let's build it


In [6]:
W = np.zeros((env.num_contexts, env.num_actions))

epsilon = 0.1
num_steps = 2000
#alpha = 0.06
history_linear = []

for step in range(num_steps):
    # 1. Input (X): Get context (random person shows up), and represent this as 1 in our state list (feature vector)
    context_idx = np.random.randint(0, env.num_contexts)
    s = np.zeros(env.num_contexts)
    s[context_idx] = 1

    # 2. Make a prediction (y_pred) - guess the CTRs for this user - with our weights at this moment
    ctr_guess = s @ W

    # 3. Choose an action
    if np.random.rand() < epsilon:
        action_idx = np.random.randint(0, env.num_actions)
    else:
        action_idx = np.argmax(ctr_guess)

    # 4. Find the 'true' label (i.e., y in supervised learning)
    reward = env.get_reward(context_idx, action_idx)
    history_linear.append((context_idx, action_idx, reward))
    target = reward

    # 5. Calculate the 'error' for the action we took
    y_pred = ctr_guess[action_idx]   # based on our predicted ctrs, we choose an action
    error = target - y_pred           # this generates an error signal - if we were wrong

    # 6. Update the weights with gradient descent (known from supervised learning and ML)
    W[:, action_idx] += error * s # Remember that the weight that contributed to this action is found in the column 'action_idx'
    # so we update the entire column, but s is [0, 1], for example, so we only add to the weight that was active, or contributed
    # the rest gets error * 0 = 0 -> old_weight + 0 = old_weight. Simple as that.

# --- Analysis ---
print("--- Scalebandit results ---")
print(f"Final weight matrix (W):\n{np.round(W, 2)}\n")

avg_reward_linear = np.mean([example[2] for example in history_linear])
print(f"Average CTR achieved: {avg_reward_linear:.2%}")

print("\nLet's test our trained 'weights':")
s_new_viewer = np.array([1, 0])
s_subscriber = np.array([0, 1])

# The learned values should approximate the true CTRs!
# True CTRs for New Viewer: [0.6, 0.1, 0.05]
# True CTRs for Subscriber: [0.1, 0.5, 0.7]
print(f"Predictions for new viewer: {s_new_viewer @ W}")
print(f"Predictions for subscriber: {s_subscriber @ W}")

--- Scalebandit results ---
Final weight matrix (W):
[[0. 0. 0.]
 [0. 0. 0.]]

Average CTR achieved: 36.45%

Let's test our trained 'weights':
Predictions for new viewer: [0. 0. 0.]
Predictions for subscriber: [0. 0. 0.]


Woops, that was not good. Click-through is roughly only 37%. That's barely beating *Youbandit* at 33.85% and way off *Personalbandit* at 62.85%. What gives?




##### Fixing a bug: an agent with no memory
--- 
Scalebandit results with **no** <span style="color:purple">learning rate</span>
 Final weight matrix (W):

[[0. 0. 0.]
 [1. 0. 0.]]

Average CTR achieved: **37.28%**

Trained 'weights':

- Predictions for new viewer: [0. 0. 0.]
- Predictions for subscriber: [1. 0. 0.]

 ---

 Scalebandit results **with** <span style="color:purple">learning rate</span>

Final weight matrix (W):
[[0.45 0.08 0.03]
 [0.03 0.51 0.71]]

Average CTR achieved: **61.61%**

Trained 'weights':
- Predictions for new viewer: [0.45448043 0.07525964 0.03243021]
- Predictions for subscriber: [0.03078805 0.50880286 0.71027821] 

So, why is this? 

If you think back to the last lecture, we used this "sample-average" method to update our belief table. The more we saw an example and got to choose an action, the less extreme the update got. So over time, the updates was very small and incremental in the end. We had this thing `1 / action_counts[action_idx]` before the difference of the reward and the estimated value. We could essentially call this this something like a *learning rate*. This learning rate acts as a built in *memory management system*. Early on, the agent has little experience, so it has a high *learning rate*. It's pretty aggressive, and it learns what it can from new information because it doesn't know any better. Later on (high N), the agent has *a lot of experience* for that state-action pair, so it treats new information as just one more data point, make only a tiny adjustment to its already-confident belief.  

Why do we need a learning rate at all? Because it allows our beliefs to 'land' in a way (converge to the true average technically). In the end, you have to decide on something, so you should not jump around too much. A high learning rate is the equivalent of jumping around a lot - never deciding if you think these are the true values or not. 

---

So why didn't we use the same sample average method or 1/N or whatever for *Scalebandit*?

The explenation is slightly technical, and the reason is that we wanted to make our *Personalbandit* general. And more general means being able to tackle **all** problems it could potentially encouter. In our YouTube <span style="color:#6F8FAF">**environment**</span> the *hidden secret* never changed. It had the values locked down. They were written in stone so to speak. Never, ever changing. However, most real-world problems are not like that. They are what is called as **<span style="color:red">non-</span>stationary**. What if user preferences of what thumbnails they like to see change over time (e.g., a viral video because old news, so it is not interesting longer)? Or maybe you play a game, the opponent change their strategy. We want to be able to solve all kinds of problems with our method. And using the sample average method, having your *learning rate* decay to near-zero is not beneficial. Then you never update yourself! You just run some trajectories, then "Yepp, OKAY, now I know everything I ever need to know". So we want somethign that is more general, and a common approach in modern RL is to use a constant, but quite small, *learning rate*.

This make it so that the agent can "forget" old, outdated information and constantly <span style="color:#00A36C">adapt</span> to new realities, or what videos are viral at any point, at least over time. 

In [7]:
# Let's expand our error not using a learning rate and just see if this create a running average
#W[:, action_idx] += (target - prediction) * s
W[:, action_idx] += (1 - (s @ W[:, action_idx])) * s # toggle the reward between 0 and 1 to see what happens

In [8]:
W[:, action_idx]

array([1., 0.])


##### learning from single samples


Let's make this tangible. We will simulate a single learning step for our agent and print out thoughts it has as we go.

We'll focus on just one part of our its brain: the weight `W[0, 0]`. This weight is responsible for predicting the click-through rate of the '🔥 Viral thing' (action=0) when shown to a 'new viewer' (state=0).

The **ground thruth** (which the agent doen't know, the *hidden secret*) is that the real CTR is 0.6

In [7]:
true_ctr = 0.6 # from 'true probs' in our HomepageEnv

In [8]:
lr = 0.1 # the agents learning rate

In [9]:
# let's say the agent has some experience, and its current belief for W[0,0] is 0.4 - currently a bit pessimistic
current_belief = 0.4
print(f"Agent's 'starting belief' for '🔥 Viral thing' shown to a new user is: {current_belief:.2f}")

Agent's 'starting belief' for '🔥 Viral thing' shown to a new user is: 0.40


Now, imagine that we get some experience, a single sample. A "new viewer" arrives, we shown them the '🔥 Viral thing', and we just sit back and observe the outcome. Let's simulate how it could play out:

In [10]:
# re-run this cell a few times. You'll seea bout 60% of the time you get a reward of 1 and 40% you'll get 0. 
reward = 1 if np.random.randn() < true_ctr else 0
print(f"A new user arrives...")
print(f"The REALITY of this single experience was: {'WAHAY! CLICK! 🤩🥳' if reward == 1 else 'No click this time :('}")

A new user arrives...
The REALITY of this single experience was: WAHAY! CLICK! 🤩🥳



##### the agents "though process"


In [11]:
# our agent just saw as single, real-world 'reward'. Now, let's trace how it learns from this single experience
prediction = current_belief
print(f"What did I actually think? Well, I thought the likely reward of a click was about: {prediction}")

target = reward
print(f"The 'target' (i.e., this one reality) was: {target}")
print()
print(f"Therefore, I want to adjust my thinking so as to believe that I would be getting {target} all along")
error = target - prediction


What did I actually think? Well, I thought the likely reward of a click was about: 0.4
The 'target' (i.e., this one reality) was: 1

Therefore, I want to adjust my thinking so as to believe that I would be getting 1 all along


How will I update my belief?

I'll <span style="color:purple">nudge</span> my <span style="color:gray">old belief</span> in the **direction of the error**

In [12]:
# nudge the old belief in the direction of the error
nudge = lr * error
nudge # so the nudge is positive / negative (depending on what happened above): if positive - we want to think "higher of" this action

0.06

In [13]:
# What's my new belief, the agent wonders?
# Well, it's what you thought before, pluss the new, updated version of what you experienced in reality
new_belief = current_belief  + nudge
print(f"My old thinking was {current_belief} likely for this action, however, after seeing a sample of real-world experience (i.e., {reward}),\nmy new thinking is: {new_belief}")

My old thinking was 0.4 likely for this action, however, after seeing a sample of real-world experience (i.e., 1),
my new thinking is: 0.46


In [14]:
current_belief = new_belief

Therefore, we can see that the <span style="color:#FF69B4">next time the agent in the same position again</span>, choosing this thumbnail to show is now a little bit **more likely**, so *it thinks*, based on what *it has experienced in the "real world"*, that this action should have a higher probability of getting itself more <span style="color:green">rewards</span> in the future. 

That's it. This is the step-by-step process that is at the heart of many reinforcement learning algorithms. The agent makes a <span style="color:blue">prediction</span>, *compares* it to a **single sample of reality**. Then it nudges its internal beliefs to be slightly more accurate next time. 

We *know* from our ML series that it is called **stochastic gradient descent** because the nudge is based on a random sample of the real word - that single <span style="color:green">'reward'</span>. It is not exact, so we don't need to take the expectation over all over samples. That's also, in some sense, how we deal with actually just doing something practical. 

Now that you understand the logic of a single update, you can just put it inside a `for-loop` and let *Scalebandit* learn from thousands or even millions of experiences. 


#### Bonus: A lookup table is a special form of linear function approximation

Let's show this relationship. The idea is pretty huge, because it birdges to what we did last lecture. It means we didn't **throw away our old method**, we <span style="color:#088F8F">generalized it</span>. Let's think about what we just built.

1. Our *Personalbandit* from Lecture 2 used a 2D table `belief_table[context_idx, action_idx]` to look up the right user and then select the appropriate thumbnail to show.
2. Our new *Scalebandit* uses a weight matrix `W` and a list which were 0 everywhere, exept for the user which had just shown up, that index was 1 (this is called 'one-hot encoded'). Then we had our beliefs be `s @ W`.

Let's trace the prediction for a 'new viewer' (i.e., context_idx=0).

- The state vector is [1, 0]
- The prediction is [1, 0] @ [[W_00, W_01, W_02], [W_10, W_11, W_12]] -> (1, 2) x (2, 3) -> (1, 3)
- The result of this dot product is simply `[W_00, W_01, W_02]`

In [16]:
new_viewer = [1, 0] # just a one representing the type of person that has arrived at our YouTube page
subscriber = [0, 1] # the same for this type of user

In [39]:
# let's say a new user showed up, and we will trace this
state = new_viewer

In [40]:
np.random.seed = 42
W = np.random.randn(env.num_contexts, env.num_actions)
W

array([[ 0.65998573, -1.54196234, -0.05353478],
       [-1.20342247,  1.10157142,  0.96988953]])

In [44]:
# see what row we have 'looked up' in our belief table? 
state @ W

array([ 0.65998573, -1.54196234, -0.05353478])

In [46]:
# see that the dot-product is the same as indexing into our 'table' with index 0 (for 'new user' from Lecture 2)
state @ W == W[0]

array([ True,  True,  True])

In [33]:
# let's say the other user shows up (i.e., context_idx=1), then state is 
state = subscriber

In [34]:
# and what happens to the dot product now?
state @ W

array([-0.79675108, -1.49999193, -1.13451925])

In [35]:
# again, equivalent with indexing into our 2D table with index 1 (for 'subscriber' from Lecture 2)
state @ W == W[1]

array([ True,  True,  True])

Wooot Our weight matrix, `W` is behaving *exactly like our 2D belief table* from Lecture 2.

- Lookng up `belief_table[0]`, gave us the values for a 'new viewer'
- Multiplying by `s=[1,0]` gives us the first row of our belief table, which are the values that correspond to what we saw in the last lecture

They are mathematically identical!

In [75]:
# and how did we build our 2D 'belief table' in Lecture 2?
belief_table = np.random.randn(env.num_contexts, env.num_actions) # say our belief table from Lecture 2 had these numbers
belief_table

array([[ 0.16676071, -0.05388295,  0.59151965],
       [ 0.58147024, -0.70740083, -2.07200065]])

What we have just discovered is a fundamental truth: **a table is a special case of a linear model**. Therefore, a lookup table is a special cas of linear value function approximation where the features are a list with a 1 for the viewer that shows up (or "one-hot encoded" indicator vectors).

In our case, "being a new viewer" is a single feature, and "being a subscriber" is another. Because our features perfectly desribe our states and don't overlap, the linear model learns to dedicate one row of weights to *each feature*, recreating the lookup table.

So, why does this matter? If they are the same, why did we do all this work?

The reason is the because this new perspective is *far more powerful* and unlocksk one solution method to the scaling problem. 

----

Let's go back to the "exploding tables" problem:

- Contexts: User type (2), device (3), country (20). Total states = 120.
- Table approach: needs a giant table with 120 rows.

---

Now, let's think in therms of **features** for our linear model:

- Instead of a single big list of size 120, we can represent our state with three separate lists, concatinated together:
- <span style="color:#808000">User type</span>: `[1, 0]` ('new viewer')
- <span style="color:darkgray">Device</span>: `[0, 1, 0]` (for desktop)
- <span style="color:#CD7F32">Country</span>: `[0, ..., 1, 0]` (for France)
- Our <span style="color:blue">state</span>`s` would be a long binary vector (a list with some numbers) of size 2 + 3 + 20 = 25.
- Our <span style="color:purple">weight matrix</span> `W` would have the shape [25, num_actions]. It would now have **25 rows**, <span style="color:red">not</span> 120!

The model would learn separate weights for the feature "new user", the feature "is on a mobile device" and the feature "is in France". When a new user shows up and is on a mobile device in France, the <span style="color:blue">agent</span> would **add the contributions** of these learned weights together to amek a prediction.

This is what we call <span style="color:#088F8F">generalisation</span>. Our agent can learn that "mobile users" generally prefer short content, and apply that knowledge to a user from a country it has barely seen before. A lookup table could never do this. 

So, while our simple implementation was equivalent to a table, the **framework** we are building out, i.e., *function approximation* is what allows us to escape the curse of dimensionality. This is the key that will allow us to build real reinforcement learning agent that can go out into the world and solve large, interesting problems. This is also our first step towards what will eventually become **Deep Reinforcement Learning**. 


#### Bonus 2: Why havent we seen the famous Q-learning algorithm yet?

Some of you may have heard of a famous concept in reinforcement learning called Q-learning, temporal-difference learning. Is this what this is?

The short answer is **no**. The error we are using here is called a **Monte Carlo** update. 

---
Why don't we have a TD error?

In our problem:
1. we take an action
2. the episode immediately terminates
3. there is **no `S_t+1`**. There is no *next state*. The future doen't exist.

If we *tried* to write the TD target for our problem, the `V(S_t+1)` term would be zero, `Q(S_t+1, a')`or becase the game is over. The formula would collapse:

`TD Target = r + γ * 0` = `r`

So, for one-step problems like ours, the **Monte Carlo** target and the **TD target** are identical. 

In summary: we have been using a Monte Carlo-style update because our problem is a one-step MDP. We haven't needed to worry about the future yet. 

However, to solve problems like  robotics, games, energy, or anything with a sequence of steps, we must learn to handle delayed <span style="color:green">rewards</span> and estimate the value of <span style="color:#CCCCFF	">future states</span>. This will require  us to fully embrace the concept of temporal-difference learning, Bellman equations and that mysterious little gamma thing. This awaits us going forward!